<a href="https://colab.research.google.com/github/lifepopkay/Tech-Monies/blob/Data-cleaning/title_skill_eligibility_cleanup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Clean `title` & extract `skills` & `eligibility` from `jobDesc`

### Imports & Mount Gdrive

In [ ]:
#@title Libraries
# library
import warnings
import pandas as pd
import numpy as np
from google.colab import drive
import re

warnings.simplefilter(action='ignore', category=FutureWarning)
drive.mount('/content/drive')



Mounted at /content/drive


In [ ]:
df = pd.read_excel('/content/drive/MyDrive/SharedWithMe/Tech Roles DataSet /CombinedDataSet/CombinedCountryDataSet.xlsx')
df.head(1)

,title,id,company,location,link,salaryDesc,postDate,JobDesc,title scraped for,Country
0,Senior Software Engineer,job_3e1d31dda6c5cb4b,Offerzen,Lagos,/company/OfferZen/jobs/Senior-Software-Enginee...,Full-time,Just posted,The Opportunity\nOur newly formed Marketplace ...,Data Scientist,Nigeria


In [ ]:
# Get Unique records
df['rk'] = df.groupby('id').cumcount()+1
df1 = df[df.rk == 1]
# df1.reset_index(drop=True)
df1.shape

(7071, 11)

### Data Cleaning

In [ ]:
# create patterns for eligibility
eliPat = r'\n*(HS|HSD|High School Diploma|MBA|BS|MS|[Pp]h\.+[dD]\.+|[bB]achelor|[mM]aster|[uU]ndergraduate|B\.Sc\.).*[\.\n]{1}'
replace_EP = {
    'undergraduate|bs|bachelor|b.sc.': 'undergraduate',
    'ph.d.|phd': 'doctorate',
    'mba|msc|ms|master': 'postgraduate',
    'HSD|High School Diploma': 'high school'
}

# apply patterns & extract columns
# eligibility
df1['eligibility'] = df1['JobDesc'].str.extract(eliPat)
df1['eligibility'] = df1['eligibility'].str.lower().replace(replace_EP, regex=True)
print("Total {} rows with {} unique values".format(len(df1['eligibility']),len(df1['eligibility'].unique())))

Total 7071 rows with 6 unique values


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


In [ ]:
# Data Cleaning with Regex replacement for title
titPat = {
        '[dD]ot\w?[nN]et': ".Net", # if contains Dot Net, Dotnet, etc. --> .Net
        '^[Bb]usiness [Aa]nalyst[^,]': "Business Analyst", # if starts with business analyst and not follow by comma --> "Business Analyst"
        '[Ss]r\.?': 'Senior', # Replace with Senior
        '[Jj]r\.?': 'Junior', # Replace with Junior
        #'^.*Data ': 'Data ', # Replace anything before Data Analyst (Finance Advisory)
        'Financial Advisory International |USARPC ': '', # remove company name
        # '[Aa]sia [Pp]acific|APAC':'APAC,', # Standardising common job title words
        # 'AVP':'Assistant Vice President',
        # 'VP':'Vice President',
        # '[Ff]resh [Gg]raduates|[Ss]enior|[Jj]unior':'', #removal of job levels
        # 'Hong Kong|pandamall':'', # removal on common job title ignore words
        # '[Cc]orporate [Bb]anking|Top Tier Bank in HK|Banking|Bank$|Investment Bank|[Cc]ore [Bb]anking':'',
        # '[Ii]nsurance|[Ff]inancial [Ss]ervices':'',
      # Garbage value cleanups
        '\([\w\d ,\-]*\)|URGENT|\!|\-\-|\-$|\u200b|\– part':'',
        '\|.*':'', # remove everything after "|"
        ' for .*| in .*| at .*':'', # remove everything after "for" # remove everything after "for", "in", "at"
        '[Rr]emote': '', # remove remote
        ' , .*':'', # remove everything after comma
        '\[.*\]':'', # remove everything within brackets
        '\s[Jj]ob.*$':'', # remove everything after job word
        '_|\-|#.*|\(.*\)?':' ', #remove underscores & dash
        ' Pune.*$| Mumbai.*$| Hyd.*$| New Delhi.*$| Bangalore.*$': '', # remove places
        ' Marketing Measure.*$|¬£.*$| ‚Ä.*$|, New.*$| year.*$| \d+.*$| :.*$| \w+\d+.*$': '', # remove from end
        '^.*Hub |^.*\: ': '', # remove from start
      }
df1['title_cleaned'] = df1['title'].replace(titPat, regex=True).str.strip().replace({'^,|,$|\–$|\-$|,$':''},regex=True).str.strip()
df1[['title','title_cleaned']].tail(50)
print("Total {} rows with {} unique values".format(len(df1['title_cleaned']),len(df1['title_cleaned'].unique())))

Total 7071 rows with 4372 unique values


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
# Data Cleaning with Regex replacement
skillPat = {
        'the role[ ]?[\n\:·]|your role[ ]?[\n\:·]|Job Scope|trust you to|What You[\'Äô]ll Do|Job Highlight|Summary|Responsibilities[\n\:·]|Responsible for|What to expect|Job Description|you will work': "!!![Role]",
        'need to have|What You Have|you must have|You have[ ]?[\n\:·]|Your expertise|Requirement[s]?[ ]?[\n\:·]|Requirements(?! )|Job Requirement|Desired Competencies|What we seek|Who You Are|What you\'ll need|About you|Qualifications': "!!![Requirements]", 
        'What You\'ll Enjoy|Benefits|What we offer|we will offer': "!!![Benefits]", 
        'Apply if|Apply Now|Please send|Please contact|Please Provide|Join Us|Closing Statement|Application Method|is one of the leading|about us':"!!![End]"
      }

def replace_jd_keywords(text):
    for k,v in skillPat.items():
        text = re.sub(k, v, str(text),flags=re.I)
    return text
        
def split_jd(text, key):
    tmp = []
    for i in text.split('!!!'):
        if i.startswith('['+key+']'):
            txt = re.sub('^\['+key+'\][ :,;]{0,5}','', i).strip().replace('\n',' ')
            tmp.append(txt) if len(txt)>0 else None
    
    if (key=='Requirements') & (len(tmp) == 0):
        tmp.append('[NO REQ]')
        for i in text.split('!!!'):
            if (not i.startswith('[')) & len(i.strip())>0:
                tmp.append(i.strip().replace('\n',' '))
        
    return '| '.join(tmp)
                        
df1['JD_cleaned'] = df1['JobDesc'].apply(replace_jd_keywords)
df1['skills'] = df1['JD_cleaned'].apply(lambda x: split_jd(x,'Requirements')).str.strip()
#df2['JD_benefits'] = df1['JD_cleaned'].apply(lambda x: split_jd(x,'Benefits'))
#df1['skills'] = df1['JD_cleaned'].apply(lambda x: split_jd(x,'Role'))
#.replace(di, flags=re.I).str.strip()
#.replace({'^,|,$|\–$|\-$|,$':''},regex=True).str.strip()


df1[['id', 'title_cleaned', 'JD_cleaned','skills']]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,id,title_cleaned,JD_cleaned,skills
0,job_3e1d31dda6c5cb4b,Senior Software Engineer,The Opportunity\nOur newly formed Marketplace ...,". Where needed, you are producing easy to unde..."
1,job_c3e2ed6ba483528b,Data Scientist,Our client requires the skills of a Data Scien...,"¬∑ B.Sc. in Statistics, Mathematics, Engineeri..."
2,job_264deaa0926c095e,Full stack developer,\nWork with development teams and product mana...,[NO REQ]
3,job_4ba7daa713f2751c,Superintendent Pharmaceutical Officer,\n\n\n\nSuperintendent Pharmaceutical Officer\...,"Bachelor‚Äôs degree in pharmacy, At least 2 y..."
4,job_45b19f53adcf0053,Project HSE Advisor I,\n\n Company : Worley \n \n\nPrimary Location...,[NO REQ]| Company : Worley Primary Locatio...
...,...,...,...,...
8695,job_2371de9155a4f835,"Director, Product Data Science","\n\n Launched in 2019, Binance.US is the fast...",10+ years prior experience in Product Data Sci...
8696,job_0a142fff71782c92,SENIOR MACHINE LEARNING ENGINEER,Company details: \nRocket Science provides end...,Act as a trusted advisor with our customers wh...
8697,job_074ba55c1c4449cc,Ing√©nieur en Machine Learning,\n\n\n\n\nVos missions au quotidien\n\n\nSUR L...,Vous avez au moins 5 ans d'exp√©rience dans l'...
8698,job_b50aa676b9867e9b,Senior Applied Research Scientist NLP,\n\nAre you excited about working at the foref...,You are a fit for the role if| Ph.D. in a rele...


In [ ]:
df1.to_csv('/content/drive/MyDrive/SharedWithMe/Tech Roles DataSet /CombinedDataSet/John_Cleaning/cleaned_dataset.csv')

In [ ]:
df1[df1['id']=='job_2371de9155a4f835']

,title,id,company,location,link,salaryDesc,postDate,JobDesc,title scraped for,Country,rk,eligibility,title_cleaned,JD_cleaned,skills
8695,"Director, Product Data Science",job_2371de9155a4f835,Binance.US,"Vancouver, BC",/rc/clk?jk=2371de9155a4f835&fccid=b963fa42311e...,Full-time,21 days ago,"\n\n Launched in 2019, Binance.US is the fast...",Data Scientist,UK,1,10+ years prior experience in Product Data Sci...,"Director, Product Data Science","\n\n Launched in 2019, Binance.US is the fast...",Build and execute an effective strategy and ro...


### Apply NLTK

In [ ]:
from nltk.tokenize import RegexpTokenizer

regexp = RegexpTokenizer('\w+')

df1['skills_token']=df1['skills'].apply(regexp.tokenize)
df1.head(3)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,title,id,company,location,link,salaryDesc,postDate,JobDesc,title scraped for,Country,rk,eligibility,title_cleaned,JD_cleaned,skills,skills_token
0,Senior Software Engineer,job_3e1d31dda6c5cb4b,Offerzen,Lagos,/company/OfferZen/jobs/Senior-Software-Enginee...,Full-time,Just posted,The Opportunity\nOur newly formed Marketplace ...,Data Scientist,Nigeria,1,". Where needed, you are producing easy to unde...",Senior Software Engineer,The Opportunity\nOur newly formed Marketplace ...,". Where needed, you are producing easy to unde...","[Where, needed, you, are, producing, easy, to,..."
1,Data Scientist,job_c3e2ed6ba483528b,GVA Partners,Lagos,/company/GVA-Partners/jobs/Data-Scientist-c3e2...,Permanent +1,Today,Our client requires the skills of a Data Scien...,Data Scientist,Nigeria,1,"¬∑ B.Sc. in Statistics, Mathematics, Engineeri...",Data Scientist,Our client requires the skills of a Data Scien...,"¬∑ B.Sc. in Statistics, Mathematics, Engineeri...","[B, Sc, in, Statistics, Mathematics, Engineeri..."
2,Full stack developer,job_264deaa0926c095e,Horizonpay Nigeria Limited,Lagos,/company/HorizonPay-Nigeria-Limited/jobs/Full-...,"‚Ç¶700,000 a monthFull-time",Today,\nWork with development teams and product mana...,Data Scientist,Nigeria,1,[NO REQ],Full stack developer,\nWork with development teams and product mana...,[NO REQ],"[NO, REQ]"


In [ ]:
import nltk

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
from nltk.corpus import stopwords

# Make a list of english stopwords
stopwords = nltk.corpus.stopwords.words("english")

# Extend the list with your own custom stopwords
my_stopwords = ['https']
stopwords.extend(my_stopwords)

# Remove stopwords
df1['skills_token'] = df1['skills_token'].apply(lambda x: [item for item in x if item not in stopwords])
df1[['id', 'title_cleaned', 'skills_token']].head(3)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


,id,title_cleaned,skills_token
0,job_3e1d31dda6c5cb4b,Senior Software Engineer,"[Where, needed, producing, easy, understand, t..."
1,job_c3e2ed6ba483528b,Data Scientist,"[B, Sc, Statistics, Mathematics, Engineering, ..."
2,job_264deaa0926c095e,Full stack developer,"[NO, REQ]"


In [ ]:
# Remove Infrequent Words
df1['skills_string'] = df1['skills_token'].apply(lambda x: ' '.join([item for item in x if len(item)>10]))
df1[['id', 'skills_string']]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,id,skills_string
0,job_3e1d31dda6c5cb4b,development Architectures environment accounta...
1,job_c3e2ed6ba483528b,Mathematics Engineering Intelligence Prescript...
2,job_264deaa0926c095e,
3,job_4ba7daa713f2751c,Outstanding information communication Negotiat...
4,job_45b19f53adcf0053,Experienced consultants construction transform...
...,...,...
8695,job_2371de9155a4f835,visualizations regressions experimentation com...
8696,job_0a142fff71782c92,regressions environments architecture Obligato...
8697,job_074ba55c1c4449cc,utilisation apprentissage automatique explorat...
8698,job_b50aa676b9867e9b,engineering prototyping coordinating demonstra...


In [ ]:
from nltk.tokenize import word_tokenize

# all words
all_words = ' '.join([word for word in df1['skills_string']])
tokenized_words = word_tokenize(all_words)

# frequency
from nltk.probability import FreqDist

fdist = FreqDist(tokenized_words)
fdist

FreqDist({'development': 4207, 'communication': 2416, 'environment': 2370, 'understanding': 2078, 'information': 1839, 'applications': 1816, 'technologies': 1730, 'application': 1530, 'requirements': 1360, 'opportunity': 1332, ...})

In [ ]:
df1['skills_string_fdist'] = df1['skills_string'].apply(lambda x: ' '.join([item for item in x if fdist[item] >= 1 ]))
from nltk.stem import WordNetLemmatizer

wordnet_lem = WordNetLemmatizer()

df1['skills_string_lem'] = df1['skills_string_fdist'].apply(wordnet_lem.lemmatize)

# check if the columns are equal
df1['is_equal']= (df1['skills_string_fdist']==df1['skills_string_lem'])

# show level count
df1.is_equal.value_counts()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

True    7071
Name: is_equal, dtype: int64

In [ ]:
df1[['id', 'skills_string', 'skills_string_lem', 'is_equal']].head()

,id,skills_string,skills_string_lem,is_equal
0,job_3e1d31dda6c5cb4b,development Architectures environment accounta...,,True
1,job_c3e2ed6ba483528b,Mathematics Engineering Intelligence Prescript...,,True
2,job_264deaa0926c095e,,,True
3,job_4ba7daa713f2751c,Outstanding information communication Negotiat...,,True
4,job_45b19f53adcf0053,Experienced consultants construction transform...,,True


In [ ]:
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist

all_words_lem = ' '.join([word for word in df1['skills_string_lem']])

words = nltk.word_tokenize(all_words_lem)
fd = FreqDist(words)

fd

FreqDist({})

In [ ]:
pd.DataFrame.from_dict(fdist)

ValueError: ignored